In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
from sklearn.decomposition import PCA
from Jvis import JTSNEBASE, JTSNE
from sklearn.cluster import KMeans
from anndata import AnnData
import scanpy as sc
#from joint_metrics import *

In [ ]:
expr_mat = pd.read_csv("./GSE126074_CellLineMixture_SNAREseq_cDNA_counts.tsv", sep="\t")

Pre-Processing

In [ ]:
scale_factor= 1000
print(expr_mat)
expr_mat = expr_mat.values
print(expr_mat)
expr_mat_log_t = np.log(scale_factor*expr_mat+1)

In [ ]:
expr_reduced = PCA(n_components=20).fit_transform(expr_mat_log_t.T)

In [ ]:
joint_tsne= TSNE(init="random").fit_transform(expr_reduced)

In [ ]:
adata = AnnData(X=expr_reduced)
sc.pp.neighbors(adata, use_rep="X")
sc.tl.louvain(adata, resolution=0.2, key_added="louvain")
louv_labels = np.array(adata.obs["louvain"].tolist())
louv_labels = [int(x) for x in louv_labels]

In [ ]:
np.unique(louv_labels)

In [2]:
vis1 = plt(joint_tsne[:,0], joint_tsne[:,1], s=3, c=louv_labels, alpha=0.8)
vis1.xlabel("TSNE-1")
vis1.ylabel("TSNE-2")
vis1.show()

In [ ]:
def make_noise(expression_matrix, prop, rseed):
    np.random.seed(rseed)
    n_rows,n_cols =expression_matrix.shape
    n_elem=np.round(n_cols*0.8)
    s = np.arange(n_cols)
    for i in range(n_rows):
        row_id = list(np.random.choice(s, size=n_elem, replace=False))
        v = expression_matrix[i, row_id]
        np.random.shuffle(v)
        expression_matrix[i, row_id] = v

In [ ]:
louv_labels=np.array(louv_labels)


In [ ]:
cl1=0
cl2=1
selected_cells = (louv_labels==cl1) + (louv_labels==cl2)
expr_matsub = expr_mat[:, selected_cells]
make_noise(expression_matrix = expr_matsub, prop = 0.8, random_seed = 0)
expr_copy = np.copy(expr_mat)                     
expr_copy[:, selected_cells] = expr_matsub

In [ ]:
expr_matsub.shape,expr_copy.shape

In [ ]:
scale_factor=1000
expr_mat_log_t = np.log(scale_factor*expr_copy + 1)
expr_reduced = PCA(n_components=20).fit_transform(expr_mat_log_t)
tsne = TSNE(init="random").fit_transform(expr_reduced)

In [ ]:
from matplotlib.colors import ListedColormap
colours = ListedColormap(['r','b','g', 'orange'])
classes = ['H1', 'BJ', 'K562', 'GM12878']

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.scatter(tsne[:,0], tsne[:,1], s=3, c = louv_labels, alpha=0.8, cmap=colours)
ax.set_xlabel('tSNE-1')
ax.set_ylabel('tSNE-2')
ax.set_title('Noise RNA')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.savefig('/data/hoan/plots/snare_tsne_noiseRNA.png', bbox_inches='tight')
plt.show()

ATAC-Seq

In [ ]:
atac_topic_mat = pd.read_csv("./GSE126074_CellLineMixture_SNAREseq_chromatin_topics.tsv", sep="\t")

In [ ]:
atac_reduced = PCA(n_components=10).fit_transform(atac_topic_mat.values.T)

In [ ]:
atac_reduced.shape

In [ ]:
joint_tsne = TSNE(init="random").fit_transform(atac_reduced)
plt.figure(figsize=(4, 3))
plt.scatter(joint_tsne[:,0], joint_tsne[:,1], s=3, c = louv_labels)
plt.xlabel('tSNE-1')
plt.ylabel('tSNE-2')
plt.show()

Random Modality

In [ ]:
noise_matrix = 100*np.random.rand(1047,20)

In [ ]:
tsne_noisemodality = TSNE(init='pca').fit_transform(noise_matrix)

In [ ]:
noise_tsne = TSNE(init="random").fit_transform(tsne_noisemodality)
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.scatter(tsne[:,0], tsne[:,1], s=3, c = louv_labels, alpha=0.8, cmap=colours)
ax.set_xlabel('tSNE-1')
ax.set_ylabel('tSNE-2')
ax.set_title('Noise RNA')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.savefig('/data/hoan/plots/snare_tsne_noiseRNA.png', bbox_inches='tight')
plt.show()

Joint Visualisation

In [ ]:
data = {"dna": expr_reduced, "chromatin": atac_reduced}
joint_tsne_obj = JTSNE(init="pca")
joint_tsne = joint_tsne_obj.fit_transform(X = data)

In [ ]:
# kNN = 10
# KNI_metrics = KNI(joint_tsne, louv_labels, kNN)
# CARI_metrics = CARI(joint_tsne, louv_labels)
# print("KNI: ", KNI_metrics, ", CARI: ", CARI_metrics)

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
ax.set_xlabel("tsne-1")
ax.set_ylabel("tsne-2")
ax.set_label("Joint 2 modal")
ax.spines["top"].set_visible=False
ax.spines["right"].set_visible=False
ax.xaxis.set_ticks_position("bottom")
ax.yaxis.set_ticks_position("left")

Joint Visualisation 3 modalities

In [ ]:
data = {"dna": expr_reduced, "chromatin": atac_reduced, "noise": noise_matrix}
joint_tsne_obj = JTSNE(init="pca")
joint_tsne = joint_tsne_obj.fit_transform(X = data, method="auto", _lambda = 3)

In [ ]:
fig = plt.figure(figsize=(4.3))
ax = fig.add_subplot(111)
ax.set_xlabel("tsne-1")
ax.set_ylabel("tsne-2")
ax.set_label("Joint 3 modal")
ax.spines["top"].set_visible=False
ax.spines["right"].set_visible=False
ax.xaxis.set_ticks_position("bottom")
ax.yaxis.set_ticks_position("left")